### **Init**

In [2]:
from pathlib import Path
import sys, os, platform

CWD  = Path.cwd().resolve()
ROOT = CWD if (CWD / "src").exists() else CWD.parent
if str(ROOT) not in sys.path: sys.path.append(str(ROOT))

print("ROOT:", ROOT, "| Python:", platform.python_version())

ROOT: D:\IIT BBS\Job Resources\Business Optima\pdf-agent | Python: 3.11.13


In [3]:
CHUNKS_DIR = ROOT / "data" / "chunks"
GRAPH_DIR  = ROOT / "outputs" / "graph" / "graph"
RERANK_DIR = ROOT / "outputs" / "reranker" / "title17"
TITLE17_MERGED = ROOT / "outputs" / "lora_hf" / "title17_merged"

print("ROOT:", ROOT)
print("CHUNKS_DIR:", CHUNKS_DIR.exists())
print("GRAPH_DIR:", GRAPH_DIR.exists())
print("RERANK_DIR:", RERANK_DIR.exists())
print("TITLE17_MERGED:", TITLE17_MERGED.exists())
print("Python:", platform.python_version())


ROOT: D:\IIT BBS\Job Resources\Business Optima\pdf-agent
CHUNKS_DIR: True
GRAPH_DIR: True
RERANK_DIR: True
TITLE17_MERGED: True
Python: 3.11.13


### **Checking prompts loader**

In [4]:
from src.serve.prompt_loader import load_prompt

for name in ["system_router","system_title17","output_enhancer","style_rules","answer_with_citations","summarize_tree"]:
    p = load_prompt(ROOT, name)
    print(f"[OK] {name}: {len(p)} chars")

[OK] system_router: 538 chars
[OK] system_title17: 549 chars
[OK] output_enhancer: 746 chars
[OK] style_rules: 667 chars
[OK] answer_with_citations: 409 chars
[OK] summarize_tree: 670 chars


### **Verifying retriever alone**

In [5]:
from src.serve.retriever import GraphRetriever, prepare_contexts
from pathlib import Path

retr = GraphRetriever(ROOT/"data/chunks", ROOT/"outputs/graph/graph", ROOT/"outputs/reranker/title17")

q = "Summarize § 114 performance rights caveat. End with [pp. 67–88]."
hits = retr.search(q, k_nodes=50, k_final_nodes=8, k_each_node=14, k_final_chunks=6)
print("hits:", len(hits))
for i,h in enumerate(hits[:3],1):
    print(f"#{i} score={h['score']:.3f} node={h['node_id']} chunk={h['chunk_id']}")
ctxs = prepare_contexts(hits, max_chars=500)
print("\nCTX[0]:\n", ctxs[0][:400] if ctxs else "NO CTX")

d:\Anaconda\envs\pdf-agent-2\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


hits: 6
#1 score=0.507 node=SEC-00017 chunk=title17-h-342
#2 score=-0.890 node=SEC-00019 chunk=title17-h-689
#3 score=-1.469 node=SEC-00016 chunk=title17-h-328

CTX[0]:
 (untitled)
§ 114 · Scope of exclusive rights in sound recordings 48
[chunk title17-h-342]


### **Title-17 HF wrapper directly**

In [6]:
from src.agent.llms import load_title17_hf
from langchain.schema import SystemMessage, HumanMessage

llm = load_title17_hf(TITLE17_MERGED)
SYS = "You answer strictly using the provided CONTEXTS. Cite pages if present. If not in contexts, say you don't know."
question = "Summarize § 114 performance rights caveat. End with [pp. 67–88]."
ctx_text = "\n\n".join(ctxs[:3]) if ctxs else "NO CONTEXTS"
user = f"CONTEXTS:\n{ctx_text}\n\nUSER QUESTION:\n{question}\n\nAnswer:"
msg = llm.invoke([SystemMessage(content=SYS), HumanMessage(content=user)])
print(type(msg), getattr(msg, "content", msg))

D:\IIT BBS\Job Resources\Business Optima\pdf-agent\src\agent\llms.py:13: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  intro_llm  = ChatOllama(model=OLLAMA_MODEL, temperature=0.3)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
D:\IIT BBS\Job Resources\Business Optima\pdf-agent\src\agent\llms.py:49: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=gen_pipe)
`generation_config` default values have been modified to match model-specific defaults: {'do_sample': True}. If this is not desired, please set these values explicitly.


<class 'str'>  The exclusive rights granted under section 106 are subject to a royalty payment requirement for certain performances, as specified in paragraph (1).


### **Verifying Ollama models reachability (router/polisher)**

In [7]:
from langchain_community.chat_models import ChatOllama
print("Trying ChatOllama (mistral:instruct)...")
try:
    test_llm = ChatOllama(model="mistral:instruct", temperature=0.1)
    print(test_llm.invoke("Say: 'router alive'").content)
except Exception as e:
    print("Ollama error:", e, "\nEnsure `ollama serve` is running and the model is pulled.")


Trying ChatOllama (mistral:instruct)...
 Router Alive


### **Router only**

In [8]:
from src.serve.orchestrator import run_chat
print("Router GENERAL ->", run_chat("hey there!", "sess1")[:120], "...")
print("Router TITLE17 ->", run_chat("Summarize §114 performance right caveat.", "sess1")[:160], "...")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 

### **Full Title-17 pass**

In [11]:
q = "Summarize § 114 performance rights caveat. End with [pp. 67–88]."
print(run_chat(q, "sess2"))


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


- Clarity and flow improved:
   - Performing a work publicly under Section 114 comes with specific conditions and exceptions.
   - Public performances via phonorecords require royalty payments to copyright owners, whereas private home viewing use is exempt.
- Typos fixed: none identified in the provided text.
- Conciseness maintained: the original text was concise.
- Bullet points preserved: as requested.
- No new claims or citations added: the draft already contains all necessary information.
- Lightly cleaned version returned: no significant changes made beyond improving clarity and flow.


### **Inspect last messages from sqlite**

In [12]:
try:
    from src.serve.sqlite_store import get_last_messages
    msgs = get_last_messages("sess2", limit=6)
    for m in msgs: print(m)
except Exception as e:
    print("sqlite not enabled or schema differs:", e)


sqlite not enabled or schema differs: get_last_messages() got an unexpected keyword argument 'limit'
